In [5]:
import json
import re
from time import sleep

from bs4 import BeautifulSoup
from selenium import webdriver
import undetected_chromedriver

# Получение ссылок на телефоны

In [6]:
url = 'https://market.yandex.ru/search?cvredirect=2&text=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD%D1%8B&local-offers-first=0&page='

In [7]:
driver = undetected_chromedriver.Chrome()

links = []
for i in range(1, 3):
    driver.get(url + str(i))
    sleep(7)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    a_tags = soup.find_all('a', class_='_2f75n _2k4aa _1_8WQ cia-cs')
    for a in a_tags:
        links.append(a.get('href'))
    print(i)

with open('links.txt', 'a') as file:
    for link in links:
        file.write(str(link) + '\n')

1
2


# Приведение ссылок на телефоны к нужному виду, устранение дубликатов

In [3]:
with open('links.txt', 'r') as file:
    text = file.read()
    a = re.findall(r'\/[A-Za-z0-9-]+\/[0-9]+\/', text)
    links_wo_repetitions = set(a)

with open('good_links.txt', 'a') as file2:
    for link in links_wo_repetitions:
        print(str(link))
        file2.write(str(link) + '\n')

/product--smartfon-xiaomi-redmi-note-10-pro/861652002/
/product--smartfon-xiaomi-redmi-9a/670614000/
/product--smartfon-realme-8/875544009/
/product--smartfon-honor-x8/1740363991/
/product--smartfon-xiaomi-11-lite-5g-ne/1449166270/
/product--smartfon-samsung-galaxy-a52/865640004/


# Парсинг отзывов в JSON

In [7]:
def get_divs():
    sleep(randint(4, 6))
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    return soup.find_all('div', class_='_3K8Ed')

In [8]:
%%time
from random import randint

with open('good_links.txt', 'r') as file:
    links = file.read().split('\n')

driver = undetected_chromedriver.Chrome()

total_reviews = 0
for link in links:
    i = 1
    while(True):
        driver.get(f'https://market.yandex.ru/{link}reviews?track=rev_mc&page={i}')
        divs = get_divs()

        if len(divs) == 0:
            if re.findall('captcha', driver.current_url):
                checkbox = driver.find_element_by_class_name('CheckboxCaptcha-Button')
                webdriver.ActionChains(driver).double_click(checkbox).perform()

                divs = get_divs()
            else:
                break

        for tag in divs:
            review_rating = tag.find('div', class_='_3iy4z _3DD8b autotest-RatingStars').get('data-rate')
            review = tag.find('div', class_='_3IXcz').find_all('dd', class_='_1yULR')
            review_text = ''
            for element in review:
                review_text += (element.text + ' ')

            with open('train_yandex_market.json', 'a', encoding='utf-8') as f:
                json.dump({'text': review_text, 'rating': review_rating}, f, ensure_ascii=False)
                f.write('\n')

        total_reviews += len(divs)
        print(f'Link: {link}\nPage: {i}\nTotal: {total_reviews}')
        i += 1

WebDriverException: Message: chrome not reachable
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x003DB8F3+2406643]
	Ordinal0 [0x0036AF31+1945393]
	Ordinal0 [0x0025C610+837136]
	Ordinal0 [0x00250442+787522]
	Ordinal0 [0x00250C78+789624]
	Ordinal0 [0x002524B2+795826]
	Ordinal0 [0x0024BF09+769801]
	Ordinal0 [0x0025DAC0+842432]
	Ordinal0 [0x002B3E62+1195618]
	Ordinal0 [0x002A4096+1130646]
	Ordinal0 [0x0027E636+976438]
	Ordinal0 [0x0027F546+980294]
	GetHandleVerifier [0x00649612+2498066]
	GetHandleVerifier [0x0063C920+2445600]
	GetHandleVerifier [0x00474F2A+579370]
	GetHandleVerifier [0x00473D36+574774]
	Ordinal0 [0x00371C0B+1973259]
	Ordinal0 [0x00376688+1992328]
	Ordinal0 [0x00376775+1992565]
	Ordinal0 [0x0037F8D1+2029777]
	BaseThreadInitThunk [0x7726FA29+25]
	RtlGetAppContainerNamedObjectPath [0x773B7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x773B7A6E+238]
